In [ ]:
!apt-get install openjdk-8-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jdk openjdk-8-jdk-headless
  openjdk-8-jre openjdk-8-jre-headless x11-utils
0 upgraded, 10 newly installed, 0 to remove and 33 not upgraded.
Need to get 40.7 MB of archives.
After this operation,

In [ ]:
import os

In [ ]:
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
!pwd

/content


In [ ]:
!echo $JAVA_HOME

/usr/lib/jvm/java-8-openjdk-amd64


In [ ]:
!pip install pyspark==2.4.4

     |████████████████████████████████| 215.7MB 54kB/s 
     |████████████████████████████████| 204kB 45.7MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130388 sha256=3e4eb0de4bf3785d1d7152010ea91c974bca3f6225f1a70c65f95ff7be3b3f2c
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


# New Section

In [ ]:
import pyspark

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
myspark = SparkSession.builder \
.master("local[*]") \
.appName("datta") \
.config("spark.executor.memory", "6g") \
.config("spark.driver.memory", "4g") \
.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
df = myspark.read.csv("/content/sample_data/california_housing_train.csv",header=True)

In [ ]:
df.show(5,False)

+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|longitude  |latitude |housing_median_age|total_rooms|total_bedrooms|population |households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|-114.310000|34.190000|15.000000         |5612.000000|1283.000000   |1015.000000|472.000000|1.493600     |66900.000000      |
|-114.470000|34.400000|19.000000         |7650.000000|1901.000000   |1129.000000|463.000000|1.820000     |80100.000000      |
|-114.560000|33.690000|17.000000         |720.000000 |174.000000    |333.000000 |117.000000|1.650900     |85700.000000      |
|-114.570000|33.640000|14.000000         |1501.000000|337.000000    |515.000000 |226.000000|3.191700     |73400.000000      |
|-114.570000|33.570000|20.000000         |1454.000000|326.000000    |624.000000 |262.000000|1.925000     |65500.000000

In [ ]:
df.createOrReplaceTempView("parquetFile")
samplecsv = myspark.sql("SELECT * FROM parquetFile")
samplecsv.show()

+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|-114.310000|34.190000|         15.000000|5612.000000|   1283.000000|1015.000000| 472.000000|     1.493600|      66900.000000|
|-114.470000|34.400000|         19.000000|7650.000000|   1901.000000|1129.000000| 463.000000|     1.820000|      80100.000000|
|-114.560000|33.690000|         17.000000| 720.000000|    174.000000| 333.000000| 117.000000|     1.650900|      85700.000000|
|-114.570000|33.640000|         14.000000|1501.000000|    337.000000| 515.000000| 226.000000|     3.191700|      73400.000000|
|-114.570000|33.570000|         20.000000|1454.000000|    326.000000| 624.000000| 262.000000|     1.925000|    

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
table_schema = StructType([StructField('status', StringType(), True),
                     StructField('txn_id', StringType(), True),
                     StructField('trn_date',TimestampType(), True),
                     StructField('fin_nfin', StringType(), True),
                     StructField('proc_code', StringType(), True),
                     StructField('resp_cpde', StringType(), True),
                     StructField('state', StringType(), True),
                     StructField('mob_no', StringType(), True)])

In [ ]:
df1 = myspark.read.csv("/content/drive/My Drive/Divefire/mb_out_large.csv",schema=table_schema,sep=",")

In [ ]:
df1.show(1)

+---------+------------------+-------------------+--------------+--------------+---------+-----------+----------+
|   status|            txn_id|           trn_date|      fin_nfin|     proc_code|resp_cpde|      state|    mob_no|
+---------+------------------+-------------------+--------------+--------------+---------+-----------+----------+
|COMPLETED|INBbxkrb7677446468|2019-05-26 20:42:26|Non-Financials|balanceInquiry|       00|Maharashtra|9945539882|
+---------+------------------+-------------------+--------------+--------------+---------+-----------+----------+
only showing top 1 row



In [ ]:
df1.printSchema()

root
 |-- status: string (nullable = true)
 |-- txn_id: string (nullable = true)
 |-- trn_date: timestamp (nullable = true)
 |-- fin_nfin: string (nullable = true)
 |-- proc_code: string (nullable = true)
 |-- resp_cpde: string (nullable = true)
 |-- state: string (nullable = true)
 |-- mob_no: string (nullable = true)



In [ ]:
df1.createOrReplaceTempView("csvfile")
bankdata = myspark.sql("SELECT *,cast(cast(trn_date as DATE) as STRING) as trn_date_part FROM csvfile")
bankdata.show()

+---------+------------------+-------------------+--------------+-----------------+---------+--------------+----------+-------------+
|   status|            txn_id|           trn_date|      fin_nfin|        proc_code|resp_cpde|         state|    mob_no|trn_date_part|
+---------+------------------+-------------------+--------------+-----------------+---------+--------------+----------+-------------+
|COMPLETED|INBbxkrb7677446468|2019-05-26 20:42:26|Non-Financials|   balanceInquiry|       00|   Maharashtra|9945539882|   2019-05-26|
|COMPLETED|INBpmcbm7874430557|2019-05-17 05:30:54|    Financials|     fundTransfer|       00|   Maharashtra|9991940167|   2019-05-17|
|COMPLETED|INBycnbh3838184217|2019-05-19 06:52:02|    Financials|     fundTransfer|       00|   Maharashtra|9998664135|   2019-05-19|
|   FAILED|INBlevml0264925092|2019-05-18 05:53:07|    Financials|     fundTransfer|       96|   Maharashtra|9997491417|   2019-05-18|
|COMPLETED|INBkovsp0528222721|2019-05-27 19:16:44|Non-Financia

In [ ]:
bankdata.printSchema()

root
 |-- status: string (nullable = true)
 |-- txn_id: string (nullable = true)
 |-- trn_date: timestamp (nullable = true)
 |-- fin_nfin: string (nullable = true)
 |-- proc_code: string (nullable = true)
 |-- resp_cpde: string (nullable = true)
 |-- state: string (nullable = true)
 |-- mob_no: string (nullable = true)
 |-- trn_date_part: string (nullable = true)



In [ ]:
bankdata.write.mode("overwrite").partitionBy("trn_date_part").parquet("/content/datta_parquet1/out")

In [ ]:
parquetFile = myspark.read.parquet("/content/datta_parquet1/out")

In [ ]:
parquetFile.show()

+---------+------------------+-------------------+--------------+-----------------+---------+--------------+----------+-------------+
|   status|            txn_id|           trn_date|      fin_nfin|        proc_code|resp_cpde|         state|    mob_no|trn_date_part|
+---------+------------------+-------------------+--------------+-----------------+---------+--------------+----------+-------------+
|COMPLETED|INBjnyax8407463428|2019-05-11 03:35:28|Non-Financials|   balanceInquiry|       00|   Maharashtra|9935844951|   2019-05-11|
|COMPLETED|INButbxt3703655342|2019-05-11 06:36:16|Non-Financials|stopChequeRequest|       00|   Maharashtra|9906117253|   2019-05-11|
|COMPLETED|INBjuqkj4734676844|2019-05-11 20:54:17|    Financials|     fundTransfer|       00|   Maharashtra|9988826560|   2019-05-11|
|   FAILED|INBhasld6980542147|2019-05-11 13:14:38|    Financials|     fundTransfer|      200|   Maharashtra|9945420243|   2019-05-11|
|COMPLETED|INBbcepz7869807822|2019-05-11 08:28:24|Non-Financia

In [ ]:
parquetFile.filter((col("status")=="COMPLETED") & (col("state")=="Maharashtra")).show()

+---------+------------------+-------------------+--------------+-----------------+---------+-----------+----------+-------------+
|   status|            txn_id|           trn_date|      fin_nfin|        proc_code|resp_cpde|      state|    mob_no|trn_date_part|
+---------+------------------+-------------------+--------------+-----------------+---------+-----------+----------+-------------+
|COMPLETED|INBjnyax8407463428|2019-05-11 03:35:28|Non-Financials|   balanceInquiry|       00|Maharashtra|9935844951|   2019-05-11|
|COMPLETED|INButbxt3703655342|2019-05-11 06:36:16|Non-Financials|stopChequeRequest|       00|Maharashtra|9906117253|   2019-05-11|
|COMPLETED|INBjuqkj4734676844|2019-05-11 20:54:17|    Financials|     fundTransfer|       00|Maharashtra|9988826560|   2019-05-11|
|COMPLETED|INBbcepz7869807822|2019-05-11 08:28:24|Non-Financials|   balanceInquiry|       00|Maharashtra|9960547923|   2019-05-11|
|COMPLETED|INBeekju7643425665|2019-05-11 00:46:28|Non-Financials|CHANGE_MPIN_MTPIN|

In [ ]:
parquetFile.cache().count()

999999

In [ ]:
parquetFile.groupBy(["state","status"]).count().collect()

[Row(state='Rajasthan', status='COMPLETED', count=11100),
 Row(state='Andhra Pradesh', status='FAILED', count=16054),
 Row(state='Tamil Nadu', status='FAILED', count=19802),
 Row(state='Maharashtra', status='FAILED', count=100023),
 Row(state='Maharashtra', status='COMPLETED', count=399703),
 Row(state='Karnataka', status='COMPLETED', count=64246),
 Row(state='Goa', status='FAILED', count=10122),
 Row(state='Madhya Pradesh', status='FAILED', count=4026),
 Row(state='Karnataka', status='FAILED', count=15836),
 Row(state='Andhra Pradesh', status='COMPLETED', count=63938),
 Row(state='Gujarat', status='COMPLETED', count=39928),
 Row(state='Goa', status='COMPLETED', count=40274),
 Row(state='Assam', status='FAILED', count=1008),
 Row(state='West Bengal', status='COMPLETED', count=3892),
 Row(state='Punjab', status='FAILED', count=195),
 Row(state='Haryana', status='FAILED', count=1013),
 Row(state='Assam', status='COMPLETED', count=3950),
 Row(state='Madhya Pradesh', status='COMPLETED', co

In [ ]:
parquetFile.unpersist()

DataFrame[status: string, txn_id: string, trn_date: timestamp, fin_nfin: string, proc_code: string, resp_cpde: string, state: string, mob_no: string, trn_date_part: date]

In [ ]:
myspark.stop()